<a href="https://colab.research.google.com/github/devnikhita/WEBATHON/blob/main/Bert_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import csv
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Function to clean CSV files
def clean_csv(input_path, output_path, expected_fields=20, drop_columns=None):
    with open(input_path, newline='', encoding='utf-8') as csvfile, open(output_path, "w", newline='', encoding='utf-8') as outputfile:
        reader = csv.reader(csvfile)
        writer = csv.writer(outputfile)
        for row in reader:
            if len(row) == expected_fields:
                writer.writerow(row)
    df = pd.read_csv(output_path)
    if drop_columns:
        df.drop(columns=drop_columns, inplace=True)
    df.replace(['', ' '], np.nan, inplace=True)
    df.dropna(inplace=True)
    return df


In [3]:
# Define columns to drop
columns_to_drop = ['Rating','PublishMonth','PublishDay','RatingDist5','RatingDist4',
                  'RatingDist3','RatingDist2','RatingDist1','RatingDistTotal',
                  'CountsOfReview','Language','pagesNumber','Count of text reviews']

In [4]:
# Clean both CSV files
df1 = clean_csv("/content/book1100k-1200k.csv", "/content/cleaned1.csv", drop_columns=columns_to_drop)
df2 = clean_csv("/content/book1200k-1300k.csv", "/content/cleaned2.csv", drop_columns=columns_to_drop)

In [5]:
# Concatenate the dataframes
df = pd.concat([df1, df2], ignore_index=True)

# Optionally, sample a subset for faster processing
df_sampled = df.sample(frac=0.1, random_state=42).reset_index(drop=True)

In [6]:
# Handle missing values if any (already done in clean_csv)
# Extract TF-IDF features
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_features = tfidf_vectorizer.fit_transform(df_sampled['Description']).toarray()

In [7]:
# Load BERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Add labels based on PublishYear
df_sampled['label'] = df_sampled['PublishYear'].apply(lambda x: 0 if x < 2000 else 1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [8]:
# Define a custom Dataset
class BookDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        encoding = self.tokenizer(
            item['Description'],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        label = torch.tensor(item['label'], dtype=torch.long)
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': label
        }

In [9]:
# Create the dataset and a subset for training
dataset = BookDataset(df_sampled, tokenizer)
train_subset = Subset(dataset, range(1000))  # Adjust the range as needed

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',            # Directory to save model checkpoints and logs
    per_device_train_batch_size=8,     # Adjust batch size as needed
    gradient_accumulation_steps=4,
    num_train_epochs=3,                # Increased epochs for better training
    logging_dir='./logs',              # Directory to save logs
    logging_steps=100,
    evaluation_strategy="steps",
    save_steps=500,
    fp16=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
# Load pre-trained DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset
)

# Train the model
trainer.train()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


TrainOutput(global_step=93, training_loss=0.593517344485047, metrics={'train_runtime': 10580.1395, 'train_samples_per_second': 0.284, 'train_steps_per_second': 0.009, 'total_flos': 394222978400256.0, 'train_loss': 0.593517344485047, 'epoch': 2.976})

In [11]:
import pickle

model_save = '/content/bert_model.pkl'
with open(model_save, 'wb') as f:
  pickle.dump(model.state_dict(), f)